<h1><center>Laboratorio 5: Benchmark Bayesiano 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Diego Irarrázaval

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Diego Canales R.

### **Link de repositorio de GitHub:** `https://github.com/DiegoCanalesR/MDS7202/blob/main/Laboratorio5.ipynb`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Objetivos principales del laboratorio](#Objetivos-principales-del-laboratorio)

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 4/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Obtener datos desde Reddit y visualizar cuales posts son más probables que sean puntuados positivamente.
- Aplicar un atajo bayesiano para obtener la mean posterior de datos.
- Optimizar a través de librerías funciones.

#Importamos librerias utiles 😸

In [1]:
%%capture
!pip install praw
!pip install line_profiler
import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://styles.redditmedia.com/t5_2rer8/styles/bannerBackgroundImage_6o2td1zc54671.jpg?width=4000&format=pjpg&s=600bcf8560dff264f1cc7c0785ba5f6d529e0c28" width="10000">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema Bayesiano, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a las subfunciones ser ejecutadas (por linea de código, vean las clases). Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [x] Estudiar la función propuesta por el equipo docente.
- [x] Estudiar los tiempos de ejecución del Código a través de un perfilador.

In [2]:
def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
  reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
                     client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
                     password='ClasesMDS7202',
                     user_agent='Clases',
                     username='DocenciaDataScience', check_for_async=False)
  subreddit  = reddit.subreddit(nombre_subreddit)

  votes, post, url = {}, {}, {}
  top_submissions = list(subreddit.hot(limit = n_hot))
  for it, top_n in enumerate(range(50, len(top_submissions),50)):
    top_n_submissions = top_submissions[:top_n]
    upvotes, downvotes, url[it], post[it] = [], [], [], []

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
  return votes, post, url

In [3]:
%load_ext line_profiler

In [4]:
%lprun -f praw_reddit praw_reddit()


`Timer unit: 1e-06 s  |
Total time: 15.7763 s  | 
Function: praw_reddit at line 1`


     Line #   Hits        Time   Per Hit   % Time Line Contents
     1                                           def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
     2         1          3.0      3.0      0.0    reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
     3         1          1.0      1.0      0.0                       client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
     4         1          1.0      1.0      0.0                       password='ClasesMDS7202',
     5         1          1.0      1.0      0.0                       user_agent='Clases',
     6         1        499.0    499.0      0.0                       username='DocenciaDataScience', check_for_async=False)
     7         1         35.0     35.0      0.0    subreddit  = reddit.subreddit(nombre_subreddit)
     8                                           
     9         1          2.0      2.0      0.0    votes, post, url = {}, {}, {}
    10         1   15690513.0 15690513.0     99.5  top_submissions = list(subreddit.hot(limit = n_hot))
    11        20         32.0      1.6      0.0    for it, top_n in enumerate(range(50, len(top_submissions),50)):
    12        19        130.0      6.8      0.0      top_n_submissions = top_submissions[:top_n]
    13        19        109.0      5.7      0.0      upvotes, downvotes, url[it], post[it] = [], [], [], []
    14                                           
    15      9519       8250.0      0.9      0.1      for submission in top_n_submissions:
    16      9500       7951.0      0.8      0.1          try:
    17      9500      11478.0      1.2      0.1              ratio = submission.upvote_ratio
    18      9500      14370.0      1.5      0.1              ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
    19      9500       9291.0      1.0      0.1              upvotes.append(ups)
    20      9500       9577.0      1.0      0.1              downvotes.append(ups - submission.score)
    21      9500      11257.0      1.2      0.1              post[it].append(submission.title)
    22      9500      11322.0      1.2      0.1              url[it].append(submission.url)
    23                                                   except Exception as e:
    24                                                       continue
    25        19       1461.0     76.9      0.0      votes[it] = np.array([upvotes, downvotes]).T
    26         1          2.0      2.0      0.0    return votes, post, url

**Respuesta:**

El proceso que notoriamente toma más tiempo es la llamada a la API de reddit consultando por los n hot reddits en el subreddit, con un 99.5% del tiempo de procesamiento. Sería posible mejorar el desempeño de la función aplicando técnicas de caching, de forma que en nuevas ejecuciones no sea necesario volver a hacer el request a reddit y se utilicen los resultados ya cargados en memoria.

## 1.2 Análisis de Tiempo con Cache

Sin duda, un factor clave en la mejora del tiempo de ejecución de una aplicación es el uso eficiente de la memoria. Por lo que es importante, que ustedes respondan las siguientes preguntas:
1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [x] Responder preguntas.
- [ ] Mejorar el código con cache.


**Respuestas Teóricas:**

1. La memoria caché es un búfer de memoria que permite almacenar resultados de operaciones, o datos en general, de forma temporal y consultarlos rápidamente. Las siglas LRU significan Least Recently Used y hacen referencia a que se eliminan primero de la memoria caché las entradas que han sido menos utilizadas recientemente. 

2.  Los costos tienen están relacionados con la consulta de datos que ya han sido eliminados de la memoria caché, pues al revisar en ella y no encontrar lo buscado (miss), se aumenta el tiempo de búsqueda.

3. Al aplicar caching en la función anterior, la consecuencia debiese ser una reducción en el tiempo de ejecución, ya que información como la obtenida al consultar por los n hot reddits se almacenaría en caché para volver a ser consultada.

In [5]:
import timeit

n = 10

if __name__ == "__main__":
    time_praw = timeit.timeit("praw_reddit()","from __main__ import praw_reddit", number=n)

print('Tiempos sobre 10 ejecuciones.')
print(f'Tiempo total sin caching: {time_praw}')
print(f'Tiempo primedio sin caching: {time_praw/n}')


Tiempos sobre 10 ejecuciones.
Tiempo total sin caching: 193.31711576499947
Tiempo primedio sin caching: 19.33171157649995


In [6]:
@lru_cache(maxsize=1000)
def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
  reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
                     client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
                     password='ClasesMDS7202',
                     user_agent='Clases',
                     username='DocenciaDataScience', check_for_async=False)
  subreddit  = reddit.subreddit(nombre_subreddit)

  votes, post, url = {}, {}, {}
  top_submissions = list(subreddit.hot(limit = n_hot))
  for it, top_n in enumerate(range(50, len(top_submissions),50)):
    top_n_submissions = top_submissions[:top_n]
    upvotes, downvotes, url[it], post[it] = [], [], [], []

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
  return votes, post, url

if __name__ == "__main__":
    time_praw_lru = timeit.timeit("praw_reddit()","from __main__ import praw_reddit", number=n)

print('Tiempos sobre 10 ejecuciones.')
print(f'Tiempo total sin caching: {time_praw_lru}')
print(f'Tiempo primedio sin caching: {time_praw_lru/n}')

Tiempos sobre 10 ejecuciones.
Tiempo total sin caching: 19.80370289200073
Tiempo primedio sin caching: 1.980370289200073


Efectivamente se observa una mejora notable en los tiempos de ejecución de la función al aplicar caching LRU. Mientras que sin caching el tiempo total de promedio sobre 10 ejecuciones es de aproximadamente 19.3 segundos, al usar caching este se reduce a casi un 10% del tiempo por ejecución, obteniéndose inclusive que el tiempo total para las 10 ejecuciones tarda solo 0.5 segundos más que sólo una ejecución sin caching.

## 1.3 Obtención de Mean Posterior y Standard Error

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia Bayesiana. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$
$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [63]:
import math

def intevalos(votos_positivos, votos_negativos):
    a = []
    b = []
    for i in range(len(votos_positivos)):
        u = votos_positivos[i]
        d = votos_negativos[i]
        a.append(1+u)
        b.append(1+d)
    
    mu = []
    sigma = []
    for j in range(len(a)):
        mu_j = a[j] / (a[j] + b[j])
        mu.append(mu_j)

        sigma_j = 1.65 * math.sqrt(a[j]*b[j] / ((a[j] + b[j])**2 * (a[j] + b[j] + 1)))
        sigma.append(sigma_j)

    return mu, sigma 

def intevalos_numpy(votos_positivos, votos_negativos):
    a = 1 + votos_positivos
    b = 1 + votos_negativos

    mu = a / (a+b)
    sigma = 1.65 * np.sqrt(a*b / ((a + b)**2 * (a + b + 1)))

    return mu, sigma


## 1.4 Comparación de rendimiento

Creadas las funciones de la sección 1.3, de forma iterativa compare los diferentes batches de datos que contiene el diccionario `votes`. Para esto genere un gráfico utilizando plotly, donde se pueda observar las diferencias de tiempo que toma ejecutar las diferentes cantidades de datos. ¿es posible observar una diferencia? ¿a qué se deberá esto?.

Aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. Grafique estos tiempos y observe comente los desempeños.

In [78]:
votes, post, url = praw_reddit()

times_intevalos = []
times_intevalos_numpy = []
for i in range(len(votes)):
    pos = votes[i][:,0]
    neg = votes[i][:,1]

    time_intevalos = timeit.timeit('intevalos(pos,neg)', globals=globals(), number=1)
    times_intevalos.append(time_intevalos)

    time_intevalos_numpy = timeit.timeit('intevalos_numpy(pos,neg)', globals=globals(), number=1)
    times_intevalos_numpy.append(time_intevalos_numpy)

df1 = pd.DataFrame(columns=['Batch', 'Tiempo', 'Función'])
for i in range(len(times_intevalos)):
    df1 = df1.append({'Batch': i, 'Tiempo': times_intevalos[i], 'Función': 'python'}, ignore_index=True)
    df1 = df1.append({'Batch': i, 'Tiempo': times_intevalos_numpy[i], 'Función': 'numpy'}, ignore_index=True)

px.line(df1, x='Batch', y='Tiempo', color='Función', title='Comparación rendimiento Python v/s Numpy en intevalos()')




Sí es posible observar una diferencia, pues los tiempos de ejecución utilizando numpy son notoriamente menores a medida que se avanza en los batches de votes. Esto ocurre porque la librería numpy implementa operaciones optimizadas sobre arreglos que entregan un mejor rendimiento que operar listas en ciclos for.



In [79]:
@jit(nopython=True)
def intevalos_JIT(votos_positivos, votos_negativos):
    a = []
    b = []
    for i in range(len(votos_positivos)):
        u = votos_positivos[i]
        d = votos_negativos[i]
        a.append(1+u)
        b.append(1+d)
    
    mu = []
    sigma = []
    for j in range(len(a)):
        mu_j = a[j] / (a[j] + b[j])
        mu.append(mu_j)

        sigma_j = 1.65 * math.sqrt(a[j]*b[j] / ((a[j] + b[j])**2 * (a[j] + b[j] + 1)))
        sigma.append(sigma_j)

    return mu, sigma 

@jit(nopython=True)
def intevalos_numpy_JIT(votos_positivos, votos_negativos):
    a = 1 + votos_positivos
    b = 1 + votos_negativos

    mu = a / (a+b)
    sigma = 1.65 * np.sqrt(a*b / ((a + b)**2 * (a + b + 1)))

    return mu, sigma


times_intevalos_jit = []
times_intevalos_numpy_jit = []
for i in range(len(votes)):
    pos = votes[i][:,0]
    neg = votes[i][:,1]

    time_intevalos_jit = timeit.timeit('intevalos_JIT(pos,neg)', globals=globals(), number=1)
    times_intevalos_jit.append(time_intevalos_jit)

    time_intevalos_numpy_jit = timeit.timeit('intevalos_numpy_JIT(pos,neg)', globals=globals(), number=1)
    times_intevalos_numpy_jit.append(time_intevalos_numpy_jit)

for i in range(len(times_intevalos_jit)):
    df1 = df1.append({'Batch': i, 'Tiempo': times_intevalos_jit[i], 'Función': 'python + numba'}, ignore_index=True)
    df1 = df1.append({'Batch': i, 'Tiempo': times_intevalos_numpy_jit[i], 'Función': 'numpy + numba'}, ignore_index=True)

px.line(df1, x='Batch', y='Tiempo', color='Función', title='Comparación rendimiento Python, Numpy y Numba en intevalos()')



In [74]:
px.line(df1[df1['Batch'] > 14], x='Batch', y='Tiempo', color='Función', title='Comparación rendimiento Python, Numpy y Numba en intevalos() (últimos batches)')

Al comparar lo obtenido con lo que sucede al agregar Numba en ambos casos, se observa un comportamiento inesperado en que los valores obtenidos utilizando Numba son notablemente mayores para el primer batch de votes. No obstante, al revisar el comportamiento hacia los últimos batches, se observa un comportamiento esperado, en que Numpy+Numba muestra el mejor rendimiento, seguido por Numpy, luego por Python+Numba y finalmente el peor desempeño es obtenido al usar únicamente Python.

## 1.5 Plot de Resultados Bayesianos

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [89]:

votos = votes[len(votes)-1]
posteo = post[len(post)-1]
print("lower bounds aproximados:")
posterior_mean, std_err = intevalos_numpy(votos[:, 0], votos[:, 1])
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {'Votos (+)':votos[order[:20], 0], 'Votos (-)':votos[order[:20], 1] ,'Post':np.array(posteo)[order[:20]], 'url': np.array(url[len(url)-1])[order[:20]]}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post
df

lower bounds aproximados:

Top 20 post ordenador por el limite inferior:



,Votos (+),Votos (-),Post,url
0,802,0,Entrevista al covid,https://i.redd.it/tnuogzo7og371.jpg
1,765,0,1900 street scene mural in northern Chile,https://i.redd.it/5mmiiogzli371.jpg
2,720,0,"Debate presidencial 2005, pregunta clave de Ma...",https://v.redd.it/xj4g8rc5sw671
3,520,0,¿Qué hacer con Talca? Bueno...,https://i.redd.it/1tk589eq89771.png
4,508,0,Usuario de r/chile realiza pregunta a JC Rodrí...,https://v.redd.it/lkth8jvxrw371
5,497,0,Chile inaugurates Latin America's first thermo...,https://www.france24.com/en/live-news/20210608...
6,457,0,"El bombardero de Lo Prado: ""Se lo mando seco e...",https://v.redd.it/l6c86bhd2w571
7,444,0,"Aysén hoy 2, no se subieron todas juntas (Bonu...",https://www.reddit.com/gallery/o6f7lr
8,404,0,"Juan Carlos Bodoque, This Charming Man (Video ...",https://v.redd.it/wf0yqt92tv771
9,402,0,OP me concede un pedido de mapa. Se agradece. ...,https://i.redd.it/ha8v9ldtcx371.jpg


In [85]:
r_order = order[::-1][-20:]
data_dic = {'mean':posterior_mean[r_order], 'std_err':std_err[r_order], 'post':ordered_post[::-1]}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post",
                 error_x="std_err")
fig.show()

**Respuesta:**

De los resultados puede observarse que no es trivial asociar los posts de mayor mean posterior con los que al restar la cantidad de votos positivos con negativos entregue un mayor valor. Esto puede corroborarse al comparar el top 1 en lower bound de mean posterior (pos-neg = 802) con el top 20 (pos-neg = 1162). Esto confirma que la solución no resulta trivial.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>